In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
tables = ['Customer', 'Product']
for table in tables:
    display_table(table)

+-------------+-------------+
| customer_id | product_key |
+-------------+-------------+
|      1      |      5      |
|      2      |      6      |
|      3      |      5      |
|      3      |      6      |
|      1      |      6      |
+-------------+-------------+
+-------------+
| product_key |
+-------------+
|      5      |
|      6      |
+-------------+


### Write an SQL query to report the customer ids from the Customer table that bought all the products in the Product table.

```
+-------------+
| customer_id |
+-------------+
| 1           |
| 3           |
+-------------+
Explanation: 
The customers who bought all the products (5 and 6) are customers with IDs 1 and 3.
```

In [3]:
%%sql 

SELECT
    customer_id, 
    product_key 
FROM Customer 
GROUP BY customer_id, product_key;


customer_id,product_key
1,5
2,6
3,5
3,6
1,6


In [4]:
%%sql 

SELECT 
    customer_id
FROM Customer
GROUP BY customer_id;

customer_id
1
2
3


In [5]:
%%sql
-- fails a test case
SELECT 
    customer_id
FROM Customer
GROUP BY customer_id
HAVING MAX(product_key) = (SELECT MAX(product_key) FROM Product) 
    AND MIN(product_key) = (SELECT MIN(product_key) FROM Product);

customer_id
1
3


In [6]:
%%sql

SELECT 
    customer_id
FROM Customer
GROUP BY customer_id
HAVING COUNT(DISTINCT product_key) = (SELECT COUNT(DISTINCT product_key) FROM Product);

customer_id
1
3


# Using Pandas

In [7]:
customer_query = %sql SELECT * FROM Customer # type: ignore 
product_query = %sql SELECT * FROM Product # type: ignore 

customer_df = customer_query.DataFrame()
product_df = product_query.DataFrame()

display(customer_df, product_df)

,customer_id,product_key
0,1,5
1,2,6
2,3,5
3,3,6
4,1,6


,product_key
0,5
1,6


In [8]:
product_df.count()

product_key    2
dtype: int64

In [9]:
total_product_count = product_df.count()[0]
total_product_count

2

In [10]:
customer_df.groupby('customer_id').agg(product_count=('product_key', 'count'))

,product_count
customer_id,
1,2
2,1
3,2


In [11]:
grouped_df = customer_df.groupby('customer_id').agg(product_count=('product_key', 'count')).reset_index()
grouped_df

,customer_id,product_count
0,1,2
1,2,1
2,3,2


In [12]:
grouped_df.query(f"product_count == {total_product_count}")

,customer_id,product_count
0,1,2
2,3,2


In [15]:
grouped_df.query(f"product_count == {total_product_count}")['customer_id']

0    1
2    3
Name: customer_id, dtype: int64

In [13]:
result = grouped_df.query(f"product_count == {total_product_count}")[['customer_id']]
result 

,customer_id
0,1
2,3
